# Install necessary packages

In [ ]:
# !pip install --user pandas

# Imports

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

# Project hyper-parameters

In [2]:
# Hyper-parameters
LR = 1e-2
EPOCHS = 200
BATCH_SIZE = 256
EMBED_DIM = 107
N_HIDDEN_LAYERS = 3
HIDDEN_DIM = 128
DROPOUT = .5
SP_DROPOUT = .2
TRAIN_SEQUENCE_LENGTH = 107
TEST_SEQUENCE_LENGTH = 103
MODEL_VERSION = "v0.1.3"

# Load and preprocess data

## Load data

In [3]:
train_df = pd.read_json("data/train.json", lines=True)

In [4]:
test_df = pd.read_json("data/test.json", lines=True)
public_test_df = test_df.query("seq_length == 107")
private_test_df = test_df.query("seq_length == 130")

In [5]:
sample_submission_df = pd.read_csv("data/sample_submission.csv")

## Preprocess data

In [6]:
symbols = "().ACGUBEHIMSX"
feat_cols = ['sequence', 'structure', 'predicted_loop_type']
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']

In [7]:
def process_features(df):
    df = df.copy()
    
    sequence_sentences = np.array(df["sequence"].values.tolist())
    structure_sentences = np.array(df["structure"].values.tolist())
    loop_sentences = np.array(df["predicted_loop_type"].values.tolist())
    
    tokenizer = Tokenizer(char_level=True, filters="")
    tokenizer.fit_on_texts(symbols)
    
    sequence_tokens = tokenizer.texts_to_sequences(sequence_sentences)
    structure_tokens = tokenizer.texts_to_sequences(structure_sentences)
    loop_tokens = tokenizer.texts_to_sequences(loop_sentences)
    
    sequences = np.stack((sequence_tokens, structure_tokens, loop_tokens), axis=1)
    
    return np.transpose(sequences, (0, 2, 1)), len(tokenizer.word_index)+1

def process_labels(df):
    df = df.copy()
    
    labels = np.array(df[pred_cols].values.tolist())
    labels = np.transpose(labels, (0, 2, 1))
    
    return labels

In [8]:
x_train, vocab_size = process_features(train_df)
y_train = process_labels(train_df)

# Define and train the model

In [9]:
def lstm_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(hidden_dim, dropout=dropout, return_sequences=True,)
    )

In [10]:
def build_model(vocab_size, seq_length=TRAIN_SEQUENCE_LENGTH, pred_len=68,
                embed_dim=EMBED_DIM,
                n_hidden_layers=N_HIDDEN_LAYERS,
                hidden_dim=HIDDEN_DIM, dropout=DROPOUT, sp_dropout=SP_DROPOUT):
    inputs = tf.keras.layers.Input(shape=(seq_length, 3))

    embed = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
    
    hidden = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3])
    )
    
    hidden = tf.keras.layers.SpatialDropout1D(sp_dropout)(hidden)
    
    for x in range(n_hidden_layers):
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
    
    truncated = hidden[:, :pred_len]
    
    out = tf.keras.layers.Dense(5, activation='linear')(truncated)
    
    model = tf.keras.Model(inputs=inputs, outputs=out)
    
    return model

In [11]:
model = build_model(vocab_size)

In [12]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 107, 3)]          0         
_________________________________________________________________
embedding (Embedding)        (None, 107, 3, 107)       1605      
_________________________________________________________________
tf_op_layer_Reshape (TensorF [(None, 107, 321)]        0         
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 107, 321)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 107, 256)          460800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 107, 256)          394240    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 107, 256)         

In [13]:
class MeanColumnwiseRMSE(tf.keras.losses.Loss):
    def __init__(self, name='MeanColumnwiseRMSE'):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
        return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

In [14]:
model.compile(tf.optimizers.Adam(learning_rate=LR), loss=MeanColumnwiseRMSE())

In [15]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.01)

history = model.fit(x_train, y_train, validation_split=.1,
          batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1,
          callbacks=[
              tf.keras.callbacks.LearningRateScheduler(scheduler),
              tf.keras.callbacks.ModelCheckpoint(f"model_{MODEL_VERSION}.h5", save_best_only=True)
          ])

Epoch 1/200
9/9 [==============================] - 3s 318ms/step - loss: 0.8736 - val_loss: 0.7255
Epoch 2/200
9/9 [==============================] - 1s 87ms/step - loss: 0.6097 - val_loss: 0.6355
Epoch 3/200
9/9 [==============================] - 1s 89ms/step - loss: 0.5505 - val_loss: 0.6087
Epoch 4/200
9/9 [==============================] - 1s 90ms/step - loss: 0.5262 - val_loss: 0.5883
Epoch 5/200
9/9 [==============================] - 1s 87ms/step - loss: 0.5115 - val_loss: 0.5789
Epoch 6/200
9/9 [==============================] - 1s 88ms/step - loss: 0.5015 - val_loss: 0.5784
Epoch 7/200
9/9 [==============================] - 1s 89ms/step - loss: 0.4935 - val_loss: 0.5656
Epoch 8/200
9/9 [==============================] - 1s 87ms/step - loss: 0.4860 - val_loss: 0.5600
Epoch 9/200
9/9 [==============================] - 1s 89ms/step - loss: 0.4801 - val_loss: 0.5539
Epoch 10/200
9/9 [==============================] - 1s 89ms/step - loss: 0.4755 - val_loss: 0.5505
Epoch 11/200
9/9 [

In [22]:
model.save('saved_model')

INFO:tensorflow:Assets written to: saved_model/assets


# Fine tune the model

In [26]:
model = tf.keras.models.load_model("saved_model", compile=False)

In [27]:
model.compile(loss=MeanColumnwiseRMSE(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))

In [28]:
ft_history = model.fit(x_train, y_train, validation_split=.1,
                       batch_size=BATCH_SIZE, epochs=100, verbose=1)

Epoch 1/100
9/9 [==============================] - 3s 286ms/step - loss: 0.3156 - val_loss: 0.4688
Epoch 2/100
9/9 [==============================] - 1s 78ms/step - loss: 0.3147 - val_loss: 0.4685
Epoch 3/100
9/9 [==============================] - 1s 86ms/step - loss: 0.3140 - val_loss: 0.4686
Epoch 4/100
9/9 [==============================] - 1s 80ms/step - loss: 0.3139 - val_loss: 0.4695
Epoch 5/100
9/9 [==============================] - 1s 82ms/step - loss: 0.3141 - val_loss: 0.4692
Epoch 6/100
9/9 [==============================] - 1s 80ms/step - loss: 0.3136 - val_loss: 0.4694
Epoch 7/100
9/9 [==============================] - 1s 79ms/step - loss: 0.3135 - val_loss: 0.4694
Epoch 8/100
9/9 [==============================] - 1s 79ms/step - loss: 0.3136 - val_loss: 0.4695
Epoch 9/100
9/9 [==============================] - 1s 79ms/step - loss: 0.3133 - val_loss: 0.4696
Epoch 10/100
9/9 [==============================] - 1s 79ms/step - loss: 0.3131 - val_loss: 0.4700
Epoch 11/100
9/9 [

## Evaluate the model

In [29]:
public_test_data, _ = process_features(public_test_df)
private_test_data, _ = process_features(private_test_df)

In [30]:
model_public = build_model(vocab_size, seq_length=107, pred_len=107)
model_private = build_model(vocab_size, seq_length=130, pred_len=130)

model_public.load_weights(f"model_{MODEL_VERSION}.h5")
model_private.load_weights(f"model_{MODEL_VERSION}.h5")

In [31]:
public_preds = model_public.predict(public_test_data)
private_preds = model_private.predict(private_test_data)

# Submission

In [32]:
preds_ls = []

for df, preds in [(public_test_df, public_preds), (private_test_df, private_preds)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)
preds_df.head()

,reactivity,deg_Mg_pH10,deg_Mg_50C,deg_pH10,deg_50C,id_seqpos
0,0.585464,0.598366,0.518347,1.969975,0.706725,id_00073f8be_0
1,2.004080,3.021512,3.253051,4.197938,2.865593,id_00073f8be_1
2,1.352085,0.454490,0.557595,0.554804,0.668549,id_00073f8be_2
3,1.140306,0.992853,1.636934,1.106833,1.573088,id_00073f8be_3
4,0.843897,0.560205,0.888484,0.568688,0.876356,id_00073f8be_4


In [33]:
submission = sample_submission_df[['id_seqpos']].merge(preds_df, on=['id_seqpos'])
submission.to_csv('submission.csv', index=False)